In [ ]:
# | default_exp mkdocs

In [ ]:
# | export

import collections
import datetime
import importlib
import itertools
import multiprocessing
import os
import pkgutil
import re
import shlex
import shutil
import subprocess  # nosec: B404
import sys
import textwrap
import types
from configparser import ConfigParser
from inspect import getmembers, isfunction, isclass, ismethod, getmodule, iscoroutine
from pathlib import Path
from typing import *

import nbdev
import typer
import yaml
import nbformat
from configupdater import ConfigUpdater, Section
from configupdater.option import Option
from fastcore.shutil import move
from nbdev.doclinks import nbdev_export, NbdevLookup
from nbdev.frontmatter import FrontmatterProc, _fm2dict
from nbdev.process import NBProcessor
from nbdev.quarto import nbdev_readme
from nbdev.quarto import prepare as nbdev_prepare
from nbdev.quarto import refresh_quarto_yml
from nbdev.serve import proc_nbs
from fastcore.basics import merge
from fastcore.foundation import L
from nbdev_mkdocs._helpers.cli_doc import generate_cli_doc
from nbdev_mkdocs._helpers.utils import (
    get_value_from_config,
    set_cwd,
    raise_error_and_exit,
)
from nbdev_mkdocs._package_data import get_root_data_path
from nbdev_mkdocs.social_image_generator import _update_social_image_in_mkdocs_yml

In [ ]:
import json
import unittest.mock
from contextlib import contextmanager
from tempfile import TemporaryDirectory
from unittest.mock import patch

import numpy as np
import pytest
from fastcore.imports import IN_NOTEBOOK
from nbdev.config import nbdev_create_config
from ruamel.yaml import YAML

from nbdev_mkdocs.social_image_generator import _read_yaml_file

## Create new

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str) -> None:
    """Create a mkdocs directory in the root path.

    Args:
        root_path: The root path of the project.

    Returns:
        None

    Raises:
        typer.Exit: If the mkdocs_template path does not exist.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!"
        )
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))

Directory /tmp/tmp1h15wsd4/mkdocs created.
/tmp/tmp1h15wsd4/mkdocs/site_overrides
/tmp/tmp1h15wsd4/mkdocs/docs_overrides
/tmp/tmp1h15wsd4/mkdocs/site_overrides/main.html
/tmp/tmp1h15wsd4/mkdocs/site_overrides/partials
/tmp/tmp1h15wsd4/mkdocs/site_overrides/partials/copyright.html
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/images
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/js
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/css
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/images/compass-outline.png
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/images/default_social_logo.png
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/js/extra.js
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/js/mathjax.js
/tmp/tmp1h15wsd4/mkdocs/docs_overrides/css/extra.css


### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"

In [ ]:
assert _mkdocs_template_path.exists()

In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()

In [ ]:
print(_mkdocs_template)

# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - md_in_html
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences:
        custom_fences:
          - name: mermaid
            class: mermaid
            format: !!python/name:pymdownx.supe

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    """Get the values from the settings file

    Args:
        settings_path: The path to the settings file
        mkdocs_template: The mkdocs template to use

    Returns:
        A dictionary of the kwargs

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs

{'title': 'nbdev-mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev-mkdocs',
 'author': 'airt',
 'description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo': 'nbdev-mkdocs',
 'git_url': 'https://github.com/airtai/nbdev-mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str) -> None:
    """Create mkdocs.yml file

    Args:
        root_path: The root path of the project

    Raises:
        ValueError: If root_path is invalid or does not exists

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        raise_error_and_exit(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}"
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

#     !cat {d}/mkdocs/mkdocs.yml

File '/tmp/tmpbuowb6b7/mkdocs/mkdocs.yml' generated.


### Create summary_template.txt

In [ ]:
# | export

_summary_template = """{sidebar}
- API
{api}
- CLI
{cli}
- [Releases]{changelog}
"""


def _create_summary_template(root_path: str) -> None:
    """Create a summary template file for mkdocs.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # create mkdocs folder if necessary
        summary_template_path = Path(root_path) / "mkdocs" / "summary_template.txt"
        summary_template_path.parent.mkdir(exist_ok=True)
        # summary_template_path.yml already exists, just return
        if summary_template_path.exists():
            typer.secho(
                f"Path '{summary_template_path.resolve()}' exists, skipping generation of it."
            )
            return

        # generated a new summary_template_path.yml file
        with open(summary_template_path, "w") as f:
            f.write(_summary_template)
            typer.secho(f"File '{summary_template_path.resolve()}' generated.")
            return
    except Exception as e:
        raise_error_and_exit(
            f"Unexpected Error while creating '{summary_template_path.resolve()}': {e}"
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)
    _create_summary_template(d)

    sidebar = """- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)"""

    api = """    - [numpy.array](api/numpy/array.md)"""

    cli = """    - [my-cli](cli/my_cli.md)"""

    changelog = "(CHANGELOG.md)"

    with open(Path(d) / "mkdocs/summary_template.txt") as f:
        summary_template = f.read()
        actual = summary_template.format(
            sidebar=sidebar, api=api, cli=cli, changelog=changelog
        )

expected = """- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)
"""
print(actual)

assert actual == expected, actual

File '/tmp/tmpe12jkki8/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpe12jkki8/mkdocs/summary_template.txt' generated.
- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)



In [ ]:
# | export


def _replace_ghp_deploy_action(root_path: str) -> None:
    """Replace the default deploy action file in the .github/workflows directory with a custom one.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = get_root_data_path() / "ghp_deploy_action_template.yml"
    if not src_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
        )

    workflows_path = Path(root_path) / ".github" / "workflows"
    workflows_path.mkdir(exist_ok=True, parents=True)

    dst_path = Path(workflows_path) / "deploy.yaml"
    shutil.copyfile(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    assert not (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()
    _replace_ghp_deploy_action(d)
    assert (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()

In [ ]:
@contextmanager
def unset_env_var(name: str):
    """Unset an environment variable.

    Args:
        name: The name of the environment variable to unset.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    in_test_defined = name in os.environ
    if in_test_defined:
        original_value = os.environ.get(name)
        del os.environ[name]
    try:
        yield
    finally:
        if in_test_defined:
            os.environ[name] = original_value


def run_nbdev_new(d):
    """Run nbdev_new

    Args:
        d: the directory to run nbdev_new in

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    path = Path(".") if Path("settings.ini").exists() else Path("..")
    Path(Path(d) / "nbs").mkdir(exist_ok=True)

    # Create a sample .gitignore file
    gitignore_path = Path(d) / ".gitignore"
    sample_git_ignore = (
        "_docs/"
        "_proc/\n"
        "*.bak\n"
        "# Byte-compiled / optimized / DLL files\n"
        "__pycache__/"
    )

    with gitignore_path.open("w", encoding="utf-8") as f:
        f.write(sample_git_ignore)

    assert gitignore_path.exists()

    # copy nbdev-mkdocs/nbs/index.ipynb folder to {d}/nbs/index.ipynb
    for fname in ["index.ipynb"]:
        shutil.copyfile(path / "nbs" / fname, Path(d) / "nbs" / fname)

    # copy config files from nbdev-mkdocs/ to {d}/
    for fname in ["setup.py"]:
        shutil.copyfile(path / fname, Path(d) / fname)

    # Generate settings.ini using nbdev in {d}
    with set_cwd(d):
        nbdev_create_config(
            repo="repo",
            branch="branch",
            user="user",
            author="author",
            author_email="author@mail.com",
            description="description",
        )
        refresh_quarto_yml()

        with unset_env_var("IN_TEST"):
            nbdev_export.__wrapped__()

        nbdev_readme.__wrapped__(chk_time=True)

In [ ]:
# | export


def _update_gitignore_file(root_path: str) -> None:
    """Add the autogenerated mkdocs directories to the .gitignore file.

    Args:
        root_path: The root path of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _mkdocs_gitignore_path = get_root_data_path() / "gitignore.txt"
    with open(_mkdocs_gitignore_path, "r") as f:
        _new_paths_to_ignore = f.read()
        _new_paths_to_ignore = "\n\n" + _new_paths_to_ignore

    gitignore_path = Path(root_path) / ".gitignore"
    if not gitignore_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {gitignore_path.resolve()} does not exists!"
        )

    with open(gitignore_path, "a") as f:
        f.write(_new_paths_to_ignore)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    assert (Path(d) / "repo").exists()

    gitignore_path = Path(d) / ".gitignore"

    with open(gitignore_path) as f:
        contents = f.read()
        assert "mkdocs/docs/" not in contents
        assert "mkdocs/site/" not in contents

    _update_gitignore_file(d)

    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents

settings.ini created.
_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/


In [ ]:
# | export


def _generate_default_social_image_link(root_path: str) -> None:
    """Add default social sharing image link to the mkdocs yaml file

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        repo = get_value_from_config(root_path, "repo")
        user = get_value_from_config(root_path, "user")

        timestamp = datetime.datetime.now().timestamp()
        img_url = f"https://opengraph.githubassets.com/{timestamp}/{user}/{repo}"

        _update_social_image_in_mkdocs_yml(root_path, img_url)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    _create_mkdocs_dir(d)
    _create_mkdocs_yaml(d)
    _create_summary_template(d)
    _replace_ghp_deploy_action(d)
    _update_gitignore_file(d)

    _generate_default_social_image_link(d)

    def _f():
        with _read_yaml_file(Path(d) / "mkdocs/mkdocs.yml") as (yaml, config):
            print(config)
            assert config["extra"]["social_image"] != ""

    _f()

settings.ini created.
Directory /tmp/tmp696db5qm/mkdocs created.
File '/tmp/tmp696db5qm/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp696db5qm/mkdocs/summary_template.txt' generated.
ordereddict([('site_name', 'repo'), ('site_url', 'https://user.github.io/repo'), ('site_author', 'author'), ('site_description', 'description'), ('repo_name', 'repo'), ('repo_url', 'https://github.com/user/repo'), ('edit_uri', ''), ('copyright', '2023 onwards, author'), ('docs_dir', 'docs'), ('site_dir', 'site'), ('plugins', [ordereddict([('literate-nav', ordereddict([('nav_file', 'SUMMARY.md')]))]), 'search', ordereddict([('mkdocstrings', ordereddict([('handlers', ordereddict([('python', ordereddict([('import', ['https://docs.python.org/3/objects.inv']), ('options', ordereddict([('heading_level', 2), ('show_category_heading', True), ('show_root_heading', True), ('show_root_toc_entry', True), ('show_signature_annotations', True), ('show_if_no_docstring', True)]))]))]))]))])]), ('markdown_extensions', ['md_in

### Bringing it all together

In [ ]:
# | export


def new(root_path: str) -> None:
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Args:
        root_path: The path to the root of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)
    _create_summary_template(root_path)
    _replace_ghp_deploy_action(root_path)
    _update_gitignore_file(root_path)
    _generate_default_social_image_link(root_path)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    mkdocs_path = Path(d) / "mkdocs"

    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "site_overrides" / "main.html").exists()
    assert (mkdocs_path / "site_overrides" / "partials" / "copyright.html").exists()
    assert (mkdocs_path / "summary_template.txt").exists()

    gitignore_path = Path(d) / ".gitignore"
    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents

    def _f():
        with _read_yaml_file(Path(d) / "mkdocs/mkdocs.yml") as (yaml, config):
            print(config)
            assert config["extra"]["social_image"] != ""

    _f()

settings.ini created.
Directory /tmp/tmp932kl2ww/mkdocs created.
File '/tmp/tmp932kl2ww/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp932kl2ww/mkdocs/summary_template.txt' generated.
_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/
ordereddict([('site_name', 'repo'), ('site_url', 'https://user.github.io/repo'), ('site_author', 'author'), ('site_description', 'description'), ('repo_name', 'repo'), ('repo_url', 'https://github.com/user/repo'), ('edit_uri', ''), ('copyright', '2023 onwards, author'), ('docs_dir', 'docs'), ('site_dir', 'site'), ('plugins', [ordereddict([('literate-nav', ordereddict([('nav_file', 'SUMMARY.md')]))]), 'search', ordereddict([('mkdocstrings', ordereddict([('handlers', ordereddict([('python', ordereddict([('import', ['https://docs.python.org/3/objects.inv']), ('options', ordereddict([('heading_level', 2), ('show_category_heading', True), ('show_root_heading', True), ('show_root_toc_entry', True), ('s

## Build

### Build markdown files

In [ ]:
# | export


def _get_files_to_convert_to_markdown(cache: Path) -> List[Path]:
    """Get a list of notebooks and qmd files that require conversion to markdown format.

    Args:
        cache: The cache directory path

    Returns:
        A list of files to convert to markdown

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    exts = [".ipynb", ".qmd"]
    files = [
        f
        for f in cache.rglob("*")
        if f.suffix in exts and not any(p.startswith(".") for p in f.parts)
    ]

    return files

In [ ]:
def create_sample_qmd_file(d):
    """Create a sample qmd file

    Args:
        d: Path to the directory where the file should be created

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    qmd_str = """---
title: "Sample"
author: Sample
date: last-modified
---

## Introduction

"""
    qmd_index_path = Path(d) / "nbs" / "sample.qmd"
    with open(qmd_index_path, "w") as f:
        f.write(qmd_str)


with TemporaryDirectory() as d:
    run_nbdev_new(d)

    (Path(d) / "_proc" / "guides").mkdir(exist_ok=True)
    (Path(d) / "_proc" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "_proc" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        (Path(d) / "_proc" / f"{i}" / ".ipynb_checkpoints").mkdir(exist_ok=True)
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "_proc" / f"{i}" / f"qmd_{i}.qmd",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / ".ipynb_checkpoints" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / f"_{i}_index.ipynb",
        )

    new(d)

    with set_cwd(d):
        nbs = _get_files_to_convert_to_markdown(Path(d))

    nbs = [str(nb) for nb in nbs]
    print(nbs)

    assert f"{d}/_proc/index.ipynb" in nbs
    assert f"{d}/_proc/guides/guides_index.ipynb" in nbs
    assert f"{d}/_proc/blogs/qmd_blogs.qmd" in nbs
    assert f"{d}/_proc/Mkdocs.ipynb" in nbs

    assert f"{d}/_proc/_quarto.yml" not in nbs
    assert f"{d}/_proc/guides/.ipynb_checkpoints/guides_index.ipynb" not in nbs
    assert f"{d}/_proc/blogs/.ipynb_checkpoints/blogs_index.ipynb" not in nbs

settings.ini created.
Directory /tmp/tmpmv0bk1_g/mkdocs created.
File '/tmp/tmpmv0bk1_g/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpmv0bk1_g/mkdocs/summary_template.txt' generated.
['/tmp/tmpmv0bk1_g/_proc/Mkdocs.ipynb', '/tmp/tmpmv0bk1_g/_proc/index.ipynb', '/tmp/tmpmv0bk1_g/_proc/guides/_guides_index.ipynb', '/tmp/tmpmv0bk1_g/_proc/guides/guides_index.ipynb', '/tmp/tmpmv0bk1_g/_proc/guides/qmd_guides.qmd', '/tmp/tmpmv0bk1_g/_proc/blogs/qmd_blogs.qmd', '/tmp/tmpmv0bk1_g/_proc/blogs/_blogs_index.ipynb', '/tmp/tmpmv0bk1_g/_proc/blogs/blogs_index.ipynb', '/tmp/tmpmv0bk1_g/nbs/sample.qmd', '/tmp/tmpmv0bk1_g/nbs/index.ipynb']


In [ ]:
# | export


def _update_conditional_content_tags(text: str) -> str:
    """Update conditional content tags.

    Args:
        text: The text to update the conditional content tags in.

    Returns:
        The updated text with the conditional content tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{(?:\s*.*\.content-visible|\s*\.content-hidden)\s*(when|unless)-format\s*=\\\s*(\"|\')\s*(html|markdown)\s*\\(\"|\')\s*.*}"
    text = re.sub(
        pattern,
        lambda m: m.group(0).replace(
            m.group(1), "when" if m.group(1) == "unless" else "unless"
        ),
        text,
    )
    return text

In [ ]:
_input = """
may have some text before ::: {markdown=1 .content-visible when-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible when-format=\\"markdown\\"}
"""
expected = """
may have some text before ::: {markdown=1 .content-visible unless-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible unless-format=\\"markdown\\"}
"""

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible when-format=\\"html\\"}'
expected = ':::  { .content-visible unless-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible  unless-format=\\"html\\"}'
expected = ':::  { .content-visible  when-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ":::  { .content-visible  unless-format=\\'markdown\\'}"
expected = ":::  { .content-visible  when-format=\\'markdown\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual


may have some text before ::: {markdown=1 .content-visible unless-format=\"markdown\" style=\"text-align: center\"}

some random text 

::: {.content-visible unless-format=\"markdown\"}

:::  { .content-visible unless-format=\"html\"}
:::  { .content-visible  when-format=\"html\"}
:::  { .content-visible  when-format=\'markdown\'}


In [ ]:
_input = ":::  { .content-hidden  when-format=\\'html\\'}"
expected = ":::  { .content-hidden  unless-format=\\'html\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  when-format=\\"markdown\\" }'
expected = '::: { .content-hidden  unless-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"html\\" }'
expected = '::: { .content-hidden  when-format=\\"html\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"markdown\\" }'
expected = '::: { .content-hidden  when-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

:::  { .content-hidden  unless-format=\'html\'}
::: { .content-hidden  unless-format=\"markdown\" }
::: { .content-hidden  when-format=\"html\" }
::: { .content-hidden  when-format=\"markdown\" }


In [ ]:
# | export


def _update_mermaid_chart_tags(text: str) -> str:
    """Convert the mermaid chart tags from quarto format to markdown format.

    Args:
        text: The text to update the mermaid chart tags in.

    Returns:
        The updated text with the mermaid chart tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r"```\s*{mermaid\s*}"
    text = re.sub(pattern, "``` mermaid", text)
    return text

In [ ]:
valid_inputs = [
    "\n```{mermaid}\nflowchart LR\n ",
    "\n\n```   {mermaid}   \nflowchart LR\n ",
]
expected = ["\n``` mermaid\nflowchart LR\n ", "\n\n``` mermaid   \nflowchart LR\n "]

for n, i in enumerate(valid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


invalid_inputs = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]
expected = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]

for n, i in enumerate(invalid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


``` mermaid
flowchart LR
 


``` mermaid   
flowchart LR
 

```{ mermaid}
flowchart LR
 


```  some text {mermaid}   
flowchart LR
 


In [ ]:
# | export


def _add_markdown_attribute_to_enable_md_in_html(text: str) -> str:
    """Add markdown attribute to enable markdown in html.

    Args:
        text: The text to add the markdown attribute to

    Returns:
        The text with the markdown attribute added

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{\s*(markdown=1)?\s*"
    text = re.sub(pattern, r"::: {markdown=1 ", text)
    return text

In [ ]:
_input = """
Sample markdown Content :::  {.content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
expected = """
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
actual = _add_markdown_attribute_to_enable_md_in_html(_input)
print(actual)

assert actual == expected


Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::



In [ ]:
# | export


def _update_quarto_tags_to_markdown_format(nb_path: Path) -> None:
    """Update Quarto tags to Markdown format

    Args:
        nb_path: Path to the notebook

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with open(nb_path, "r") as f:
        contents = f.read()

    contents = _update_conditional_content_tags(contents)
    contents = _update_mermaid_chart_tags(contents)
    contents = _add_markdown_attribute_to_enable_md_in_html(contents)

    with open(nb_path, "w") as f:
        f.write(contents)

In [ ]:
def _create_test_notebook_contents() -> Dict[str, Any]:
    """Create a sample notebook using json"""
    return {
        "cells": [
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": '::: {.content-visible when-format="markdown"}\n\nWill only appear in markdown.\n\n:::\n\n```{mermaid}\nflowchart LR\n  A[Hard edge] --> B(Round edge)\n  B --> C{Decision}\n  C --> D[Result one]\n  C --> E[Result two]\n```\n',
            },
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": '::: {.content-visible when-format="html"}\n\nWill only appear in html.\n\n:::',
            },
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3",
            },
            "language_info": {
                "codemirror_mode": {"name": "ipython", "version": 3},
                "file_extension": ".py",
                "mimetype": "text/x-python",
                "name": "python",
                "nbconvert_exporter": "python",
                "pygments_lexer": "ipython3",
                "version": "3.8.3",
            },
        },
        "nbformat": 4,
        "nbformat_minor": 4,
    }

In [ ]:
test_nb_contents = _create_test_notebook_contents()

with TemporaryDirectory() as d:
    run_nbdev_new(d)
    fname = Path(d) / "test.ipynb"
    with open(fname, "w") as f:
        f.write(json.dumps(test_nb_contents))

    _update_quarto_tags_to_markdown_format(fname)

    with open(fname, "r") as f:
        contents = json.loads(f.read())


print(contents)

assert (
    '{markdown=1 .content-visible unless-format="markdown"}'
    in contents["cells"][0]["source"]
)
assert (
    not '{markdown=1 .content-visible when-format="markdown"}'
    in contents["cells"][0]["source"]
)

assert "mermaid" in contents["cells"][0]["source"]
assert not "{mermaid}" in contents["cells"][0]["source"]

assert (
    '{markdown=1 .content-visible unless-format="html"}'
    in contents["cells"][1]["source"]
)
assert (
    not '{markdown=1 .content-visible when-format="html"}'
    in contents["cells"][1]["source"]
)

settings.ini created.
{'cells': [{'cell_type': 'markdown', 'metadata': {}, 'source': '::: {markdown=1 .content-visible unless-format="markdown"}\n\nWill only appear in markdown.\n\n:::\n\n``` mermaid\nflowchart LR\n  A[Hard edge] --> B(Round edge)\n  B --> C{Decision}\n  C --> D[Result one]\n  C --> E[Result two]\n```\n'}, {'cell_type': 'markdown', 'metadata': {}, 'source': '::: {markdown=1 .content-visible unless-format="html"}\n\nWill only appear in html.\n\n:::'}], 'metadata': {'kernelspec': {'display_name': 'Python 3', 'language': 'python', 'name': 'python3'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, 'file_extension': '.py', 'mimetype': 'text/x-python', 'name': 'python', 'nbconvert_exporter': 'python', 'pygments_lexer': 'ipython3', 'version': '3.8.3'}}, 'nbformat': 4, 'nbformat_minor': 4}


In [ ]:
# | export


def _sprun(cmd: str) -> None:
    """Run a command via subprocess.check_output

    Args:
        cmd: The command to run

    Raises:
        subprocess.CalledProcessError: If the command fails

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        subprocess.check_output(
            cmd, shell=True  # nosec: B602:subprocess_popen_with_shell_equals_true
        )

    except subprocess.CalledProcessError as e:
        sys.exit(
            f"CMD Failed: e={e}\n e.returncode={e.returncode}\n e.output={e.output}\n e.stderr={e.stderr}\n cmd={cmd}"
        )

In [ ]:
# | export


def _generate_markdown_from_files(root_path: str, cache_path: Path) -> None:
    """Generate markdown files from notebook files.

    Args:
        root_path: The root path of the project.
        cache_path: The path to the _proc directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    doc_path = Path(root_path) / "mkdocs" / "docs"
    doc_path.mkdir(exist_ok=True, parents=True)

    with set_cwd(root_path):
        files = _get_files_to_convert_to_markdown(cache_path)

        for f in files:
            dir_prefix = str(f.parent)[len(str(cache_path)) + 1 :]
            dst_md = doc_path / f"{dir_prefix}" / f"{f.stem}.md"
            dst_md.parent.mkdir(parents=True, exist_ok=True)

            _update_quarto_tags_to_markdown_format(f)

            cmd = f'cd "{cache_path}" && quarto render "{f}" -o "{f.stem}.md" -t gfm --no-execute'
            _sprun(cmd)

            src_md = cache_path / "_docs" / f"{f.stem}.md"
            shutil.move(str(src_md), dst_md)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    (Path(d) / "nbs" / "guides").mkdir(exist_ok=True)
    (Path(d) / "nbs" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "nbs" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "nbs" / f"{i}" / f"qmd_{i}.qmd",
        )

    test_nbs = Path(d) / "nbs" / "test.ipynb"
    test_nb_contents = _create_test_notebook_contents()
    with open(test_nbs, "w") as f:
        f.write(json.dumps(test_nb_contents))

    new(d)
    
    with set_cwd(d):
        cache_path = proc_nbs()

    _generate_markdown_from_files(d, cache_path)

    # check markdown files
    print("Checks:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    mds = [str(md) for md in mds]
    print("\n".join(mds))
    assert len(mds) > 0, len(mds)

    for i in [
        "index.md",
        "sample.md",
        "guides/guides_index.md",
        "guides/qmd_guides.md",
        "blogs/blogs_index.md",
        "blogs/qmd_blogs.md",
        "Mkdocs.md",
    ]:
        assert f"{d}/mkdocs/docs/{i}" in mds, f"{d}/mkdocs/docs/{i}"

    with open(Path(d) / "_proc" / "test.ipynb", "r") as f:
        contents = json.loads(f.read())

print(contents)

assert '{markdown=1 .content-visible unless-format="markdown"}' in "".join(
    contents["cells"][0]["source"]
)
assert not '{markdown=1 .content-visible when-format="markdown"}' in "".join(
    contents["cells"][0]["source"]
)

assert '{markdown=1 .content-visible unless-format="html"}' in "".join(
    contents["cells"][2]["source"]
)
assert not '{markdown=1 .content-visible when-format="html"}' in "".join(
    contents["cells"][2]["source"]
)

settings.ini created.
Directory /tmp/tmpqbzq298o/mkdocs created.
File '/tmp/tmpqbzq298o/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpqbzq298o/mkdocs/summary_template.txt' generated.


/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')


Checks:
/tmp/tmpqbzq298o/mkdocs/docs/sample.md
/tmp/tmpqbzq298o/mkdocs/docs/index.md
/tmp/tmpqbzq298o/mkdocs/docs/test.md
/tmp/tmpqbzq298o/mkdocs/docs/Mkdocs.md
/tmp/tmpqbzq298o/mkdocs/docs/guides/qmd_guides.md
/tmp/tmpqbzq298o/mkdocs/docs/guides/guides_index.md
/tmp/tmpqbzq298o/mkdocs/docs/blogs/qmd_blogs.md
/tmp/tmpqbzq298o/mkdocs/docs/blogs/blogs_index.md
{'cells': [{'cell_type': 'markdown', 'metadata': {}, 'source': ['::: {markdown=1 .content-visible unless-format="markdown"}\n', '\n', 'Will only appear in markdown.\n', '\n', ':::\n', '\n', '``` mermaid\n', 'flowchart LR\n', '  A[Hard edge] --> B(Round edge)\n', '  B --> C{Decision}\n', '  C --> D[Result one]\n', '  C --> E[Result two]\n', '```']}, {'cell_type': 'markdown', 'metadata': {}, 'source': ['<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->']}, {'cell_type': 'markdown', 'metadata': {}, 'source': ['::: {markdown=1 .content-visible unless-format="html"}\n', '\n', 'Will only appear in html.\n', '\n', ':::']}], 'met

In [ ]:
# | export


def _replace_all(text: str, dir_prefix: str) -> str:
    """Replace the images relative path in the markdown string

    Args:
        text: The markdown string
        dir_prefix: Sub directory prefix to append to the image's relative path

    Returns:
        The text with the updated images relative path

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _replace = {}

    image_patterns = [
        (
            re.compile(r"!\[[^\]]*\]\(([^https?:\/\/].*?)\s*(\"(?:.*[^\"])\")?\s*\)"),
            "../images/nbs/",
        ),
        (
            re.compile(r"<img\s*src\s*=\s*\"([^http|https][^\"]*)\""),
            "../../images/nbs/",
        ),
    ]

    for pattern, image_path in image_patterns:
        matches = [match.groups()[0] for match in pattern.finditer(text)]
        if len(matches) > 0:
            for m in matches:
                _replace[m] = (
                    os.path.normpath(Path(image_path).joinpath(f"{dir_prefix}/{m}"))
                    if len(dir_prefix) > 0
                    else f"images/nbs/{m}"
                )

    for k, v in _replace.items():
        text = text.replace(k, v)

    return text

In [ ]:
text = """![Git Repo_Clone_Page](../img/test.png)
![Git Repo_Clone_Page](images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />
"""

expected = """![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../../images/nbs/images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />
"""

dir_prefix = "guides"
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected, actual

text = """![Git Repo_Clone_Page](img/test.png)
![Git Repo_Clone_Page](guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo of an opened research notebook with diagrams and writing in French" />
"""

expected = """![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/nbs/images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo of an opened research notebook with diagrams and writing in French" />
"""

dir_prefix = ""
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected, actual

![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../../images/nbs/images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />

![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/nbs/images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo o

In [ ]:
# | export


def _update_path_in_markdown(cache_path: Path, doc_path: Path) -> None:
    """Update guide images relative path in the markdown files

    Args:
        cache_path: The path to the _proc directory.
        doc_path: Path to the mkdocs/docs directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    files = _get_files_to_convert_to_markdown(cache_path)

    for file in files:
        dir_prefix = str(file.parent)[len(str(cache_path)) + 1 :]
        md = doc_path / f"{dir_prefix}" / f"{file.stem}.md"

        with open(Path(md), "r") as f:
            _new_text = f.read()
            _new_text = _replace_all(_new_text, dir_prefix)
        with open(Path(md), "w") as f:
            f.write(_new_text)


def _copy_images_to_docs_dir(root_path: str, cache_path: Path) -> None:
    # Reference: https://github.com/quarto-dev/quarto-cli/blob/main/src/core/image.ts#L38
    """Copy images from nbs to docs directory.

    Args:
        root_path: The root path of the project.
        cache_path: The path to the _proc directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    image_extensions = [
        ".apng",
        ".avif",
        ".gif",
        ".jpg",
        ".jpeg",
        ".jfif",
        ".pjpeg",
        ".pjp",
        ".png",
        ".svg",
        ".webp",
    ]

    nbs_images_path = [
        p for p in Path(cache_path).glob(r"**/*") if p.suffix in image_extensions
    ]

    if len(nbs_images_path) > 0:
        doc_path = Path(root_path) / "mkdocs" / "docs"
        img_path = Path(doc_path) / "images" / "nbs"
        for src_path in nbs_images_path:
            dir_prefix = str(src_path.parent)[len(str(cache_path)) + 1 :]
            dst_path = Path(img_path) / f"{dir_prefix}"
            dst_path.mkdir(exist_ok=True, parents=True)
            shutil.copy(src_path, dst_path)

        _update_path_in_markdown(cache_path, doc_path)

In [ ]:
def copy_guides(src, dst):
    """Copy guides from one directory to another.

    Args:
        src: Source directory
        dst: Destination directory

    Raises:
        AssertionError: If src does not exist

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src = Path(src)
    dst = Path(dst)
    assert src.exists()
    src_guides = src / "nbs" / "guides"
    src_guides_len = len(src_guides.parts)

    dst_guides = dst / "nbs" / "guides"

    for ext in [".ipynb", ".png", ".jpeg", ".jpg"]:
        for src_f in src_guides.glob(f"**/*{ext}"):
            dst_parts = dst_guides.parts + src_f.parts[src_guides_len:]
            dst_f = Path(*dst_parts)
            dst_f.parent.mkdir(exist_ok=True, parents=True)

            print(f"{src_f}, {dst_f}")
            shutil.copyfile(src_f, dst_f)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    new(d)

    with set_cwd(d):
        cache_path = proc_nbs()
    
    _generate_markdown_from_files(root_path=d, cache_path=cache_path)
    _copy_images_to_docs_dir(d, cache_path)

    # check image files
    print("Checks for images:")
    imgs = list((Path(d) / "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)

    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "Mkdocs.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "Social_Image_Generator.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "guides" / "Basic_User_Guide.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "blogs" / "Basic_User_Guide.md").exists()

    with open(
        (Path(d) / "mkdocs" / "docs" / "blogs" / "Basic_User_Guide.md"),
        "r",
    ) as f:
        contents = f.read()
        assert (
            "![Empty Git Repo](../images/nbs/blogs/images/empty_git_repo.png)"
            in contents
        )
        assert (
            "![Git Repo_Clone_Page](../images/nbs/blogs/images/git_repo_clone_page.png)"
            in contents
        )
        assert "![](../images/nbs/blogs/images/jupyter_home.png)" in contents

    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "blogs"
        / "images"
        / "jupyter_home.png"
    ).exists()
    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "guides"
        / "images"
        / "empty_git_repo.png"
    ).exists()

    print("OK.")

settings.ini created.
../nbs/guides/Setting_Up_Document_Versioning.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Setting_Up_Document_Versioning.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Auto_Generating_Docstrings.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Auto_Generating_Docstrings.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Setting_up_social_cards.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Setting_up_social_cards.ipynb
../nbs/guides/Adding_Guides.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Adding_Guides.ipynb
../nbs/guides/Adding_Release_Notes.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Adding_Release_Notes.ipynb
../nbs/guides/Advanced_Customization_Options.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Advanced_Customization_Options.ipynb
../nbs/guides/Handling_Pandas_In_The_Output.ipynb, /tmp/tmpy9r2v72i/nbs/guides/Handling_Pandas_In_The_Output.ipynb
../nbs/guides/images/cli_not_found.png, /tmp/

/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')


Checks for images:
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/enable_gh_pages.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/guide_1.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/guide_notebook.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/jupyter_home.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/releases_default.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/doc_versioning.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/foo_doc_string.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/docstring-gen-extension-btn.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/releases.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/cli_not_found.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/empty_git_repo.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/nbdev_mkdocs_preview_light_1.png
/tmp/tmpy9r2v72i/mkdocs/docs/images/nbs/guides/images/git_repo_clone_page.png
/tmp/tmpy9r2v

### Build summary for guides

In [ ]:
# | export


def _get_title_from_notebook(cache_path: Path, file_path: Path) -> str:
    """Get the title of a notebook or markdown file.

    Args:
        cache_path: The path to the _proc directory.
        file_path: The path to the notebook file.

    Returns:
        The title of the file.

    Raises:
        ValueError: If the file does not exist.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    title: str
    _file_path = cache_path / file_path

    if not _file_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {_file_path.resolve()} does not exists!"
        )

    if _file_path.suffix == ".ipynb":
        nbp = NBProcessor(_file_path, procs=FrontmatterProc)
        nbp.process()

        if "title" in nbp.nb.frontmatter_:
            title = nbp.nb.frontmatter_["title"]
        else:
            headers = [
                cell["source"]
                for cell in nbp.nb["cells"]
                if cell["cell_type"] == "markdown" and cell["source"].startswith("#")
            ]
            title = (
                f"{_file_path.stem}.html"
                if len(headers) == 0
                else headers[0].replace("#", "").strip()
            )
    else:
        with open(_file_path) as f:
            contents = f.read()
        metadata = _fm2dict(contents, nb=False)
        metadata = {k.lower(): v for k, v in metadata.items()}
        title = metadata["title"]

    return title

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    create_sample_qmd_file(d)

    new(d)

    cmd = f'cd "{d}" && nbdev_docs'
    _sprun(cmd)
    
    with set_cwd(d):
        cache_path = proc_nbs()

    _generate_markdown_from_files(d, cache_path)

    nb_paths = [
        Path("index.ipynb"),
        Path("sample.qmd"),
        Path("Mkdocs.ipynb"),
        Path("Social_Image_Generator.ipynb"),
        Path("guides/Basic_User_Guide.ipynb"),
    ]

    expected = [
        "Material for nbdev",
        "Sample",
        "Create new",
        "Social_Image_Generator.html",
        "Basic User Guide",
    ]
    actual = []
    for nb_path in nb_paths:
        actual.append(_get_title_from_notebook(cache_path, nb_path))

    print(actual)

    assert actual == expected

settings.ini created.
../nbs/guides/Setting_Up_Document_Versioning.ipynb, /tmp/tmpahqic2df/nbs/guides/Setting_Up_Document_Versioning.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /tmp/tmpahqic2df/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Auto_Generating_Docstrings.ipynb, /tmp/tmpahqic2df/nbs/guides/Auto_Generating_Docstrings.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /tmp/tmpahqic2df/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Setting_up_social_cards.ipynb, /tmp/tmpahqic2df/nbs/guides/Setting_up_social_cards.ipynb
../nbs/guides/Adding_Guides.ipynb, /tmp/tmpahqic2df/nbs/guides/Adding_Guides.ipynb
../nbs/guides/Adding_Release_Notes.ipynb, /tmp/tmpahqic2df/nbs/guides/Adding_Release_Notes.ipynb
../nbs/guides/Advanced_Customization_Options.ipynb, /tmp/tmpahqic2df/nbs/guides/Advanced_Customization_Options.ipynb
../nbs/guides/Handling_Pandas_In_The_Output.ipynb, /tmp/tmpahqic2df/nbs/guides/Handling_Pandas_In_The_Output.ipynb
../nbs/guides/images/cli_not_found.png, /tmp/

In [ ]:
# | export


def _get_sidebar_from_config(file_path: Path) -> List[Any]:
    """Get the sidebar contents from the sidebar.yml or _quarto.yml file.

    Args:
        file_path: Path to the sidebar.yml or _quarto.yml file.

    Returns:
        The sidebar contents.

    Raises:
        KeyError: If the sidebar is not defined in the config file.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    if not file_path.exists():
        raise_error_and_exit(f"Path '{file_path.resolve()}' does not exists!")

    try:
        with open(file_path) as f:
            config = yaml.safe_load(f)
        sidebar: List[Any] = config["website"]["sidebar"]["contents"]
    except KeyError as e:
        raise_error_and_exit(
            f"Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml."
        )

    return sidebar


def _read_sidebar_from_yml(root_path: str) -> List[Union[str, Any]]:
    """Get the sidebar contents from the sidebar.yml or _quarto.yml file.

    Args:
        root_path: The root path of the project.

    Returns:
        A list of strings and objects.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _proc_dir = Path(root_path) / "_proc"
    sidebar_yml_path = _proc_dir / "sidebar.yml"
    _quarto_yml_path = _proc_dir / "_quarto.yml"

    custom_sidebar = get_value_from_config(root_path, "custom_sidebar")
    if custom_sidebar == "False":
        cmd = f'cd "{root_path}" && nbdev_docs'
        _sprun(cmd)

    return (
        _get_sidebar_from_config(sidebar_yml_path)
        if sidebar_yml_path.exists()
        else _get_sidebar_from_config(_quarto_yml_path)
    )

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    create_sample_qmd_file(d)
    new(d)

    with set_cwd(d):
        sidebar = _read_sidebar_from_yml(d)
    print(sidebar)
    assert sidebar == ["index.ipynb", "sample.qmd"]

settings.ini created.
Directory /tmp/tmpmb0rp3la/mkdocs created.
File '/tmp/tmpmb0rp3la/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpmb0rp3la/mkdocs/summary_template.txt' generated.
['index.ipynb', 'sample.qmd']


In [ ]:
with pytest.raises(typer.Exit) as e:
    with TemporaryDirectory() as d:
        run_nbdev_new(d)

        new(d)

        updater = ConfigUpdater()
        updater.read(Path(d) / "settings.ini")
        updater["DEFAULT"]["custom_sidebar"] = True
        updater.update_file()
        with set_cwd(d):
            _read_sidebar_from_yml(d)

print("OK")

settings.ini created.
Directory /tmp/tmp6y0yetga/mkdocs created.
File '/tmp/tmp6y0yetga/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp6y0yetga/mkdocs/summary_template.txt' generated.


Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml.


OK


In [ ]:
# | export


def _flattern_sidebar_items(items: List[Union[str, Any]]) -> List[Union[str, Any]]:
    """Flatten a list of items.

    Args:
        items: A list of items.

    Returns:
        A flattened list of items.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    return [i for item in items if isinstance(item, list) for i in item] + [
        item for item in items if not isinstance(item, list)
    ]


def _expand_sidebar_if_needed(
    root_path: str, sidebar: List[Union[str, Any]]
) -> List[Union[str, Any]]:
    """Expand the sidebar if needed.

    Args:
        root_path: The root path of the project
        sidebar: The sidebar to expand

    Returns:
        The expanded sidebar

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _proc_dir = Path(root_path) / "_proc"
    exts = [".ipynb", ".qmd"]

    for index, item in enumerate(sidebar):
        if "auto" in item:
            files = list(_proc_dir.glob("".join(item["auto"].split("/")[1:])))  # type: ignore
            files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts])  # type: ignore
            sidebar[index] = files

        if isinstance(item, dict) and "contents" in item:
            _contents = item["contents"]
            if isinstance(_contents, str) and bool(re.search(r"[*?\[\]]", _contents)):
                files = list(_proc_dir.glob(item["contents"]))
                files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts])  # type: ignore
                item["contents"] = files

    flat_sidebar = _flattern_sidebar_items(sidebar)
    return flat_sidebar

In [ ]:
_sidebar = """- auto: \"/*.ipynb\"
- section: Blogs
  contents: blogs/*
- section: Guides
  contents: guides/*
- section: Explanations
  contents:
  - explanations/explanation_1.ipynb
  - explanations/explanation_2.ipynb
"""

expected = [
    "getting_started.ipynb",
    "index.ipynb",
    {"section": "Blogs", "contents": ["blogs/blogs_1.ipynb", "blogs/blogs_2.ipynb"]},
    {
        "section": "Guides",
        "contents": ["guides/guides_1.ipynb", "guides/guides_2.ipynb"],
    },
    {
        "section": "Explanations",
        "contents": [
            "explanations/explanation_1.ipynb",
            "explanations/explanation_2.ipynb",
        ],
    },
]


with TemporaryDirectory() as d:
    run_nbdev_new(d)

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    shutil.copyfile(
        Path(d) / "nbs" / "index.ipynb", Path(d) / "nbs" / "getting_started.ipynb"
    )

    for dir_name in ["guides", "blogs", "explanations"]:
        (Path(d) / "nbs" / dir_name).mkdir(parents=True)
        for i in [f"{dir_name}_1", f"{dir_name}_2"]:
            shutil.copyfile(
                Path(d) / "nbs" / "index.ipynb",
                (Path(d) / "nbs" / f"{dir_name}" / f"{i}.ipynb"),
            )

    new(d)

    # update _quarto.yml file
    _quarto_yml_path = Path(d) / "nbs" / "_quarto.yml"
    _yaml = YAML()
    config = _yaml.load(_quarto_yml_path)
    config["website"]["sidebar"]["contents"] = _yaml.load(_sidebar)
    _yaml.dump(config, _quarto_yml_path)

    # Set the custom_sidebar flag to True
    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["custom_sidebar"] = True
    updater.update_file()
    
    with set_cwd(d):
        cache_path = proc_nbs()

    _generate_markdown_from_files(d, cache_path)
    _copy_images_to_docs_dir(d, cache_path)

    sidebar = _read_sidebar_from_yml(d)

    actual = _expand_sidebar_if_needed(d, sidebar)

    print(actual)
    assert actual == expected, actual

settings.ini created.
Directory /tmp/tmp1md47915/mkdocs created.
File '/tmp/tmp1md47915/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp1md47915/mkdocs/summary_template.txt' generated.
['getting_started.ipynb', 'index.ipynb', {'section': 'Blogs', 'contents': ['blogs/blogs_1.ipynb', 'blogs/blogs_2.ipynb']}, {'section': 'Guides', 'contents': ['guides/guides_1.ipynb', 'guides/guides_2.ipynb']}, {'section': 'Explanations', 'contents': ['explanations/explanation_1.ipynb', 'explanations/explanation_2.ipynb']}]


In [ ]:
# | export


def _generate_nav_from_sidebar(
    sidebar_items: List[Union[str, Dict[str, Any]]], cache_path: Path, level: int = 0
) -> str:
    """Generate a navigation string for mkdocs from a sidebar list.

    Args:
        sidebar_items: A list of strings or dictionaries.
        cache_path: The path to the _proc directory.
        level: The level of indentation to use.

    Returns:
        str: The navigation string.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    output = ""
    links = [
        "{}- [{}]({}.md)\n".format(
            "    " * level,
            _get_title_from_notebook(cache_path, Path(item)),
            Path(item).with_suffix(""),
        )
        if isinstance(item, str)
        else "{}- {}\n".format("    " * level, item["section"])
        + _generate_nav_from_sidebar(item["contents"], cache_path, level + 1)
        for item in sidebar_items
    ]
    output += "".join(links)
    return output

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    for dir_name in ["guides", "blogs"]:
        (Path(d) / "nbs" / dir_name).mkdir(parents=True)
        for i in [f"{dir_name}_1", f"{dir_name}_2"]:
            shutil.copyfile(
                Path(d) / "nbs" / "index.ipynb",
                (Path(d) / "nbs" / f"{dir_name}" / f"{i}.ipynb"),
            )

    create_sample_qmd_file(d)

    new(d)
    with set_cwd(d):
        cache_path = proc_nbs()
        sidebar = _read_sidebar_from_yml(d)
        expanded_sidebar = _expand_sidebar_if_needed(d, sidebar)
        assert expanded_sidebar == [
            "index.ipynb",
            "Mkdocs.ipynb",
            "Social_Image_Generator.ipynb",
            "sample.qmd",
            {"section": "api", "contents": ["api/Mkdocs.ipynb"]},
            {
                "section": "blogs",
                "contents": ["blogs/blogs_1.ipynb", "blogs/blogs_2.ipynb"],
            },
            {
                "section": "guides",
                "contents": ["guides/guides_1.ipynb", "guides/guides_2.ipynb"],
            },
        ]

        actual = _generate_nav_from_sidebar(expanded_sidebar, cache_path)
        print(actual)

        expected = """- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- [Sample](sample.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Material for nbdev](blogs/blogs_1.md)
    - [Material for nbdev](blogs/blogs_2.md)
- guides
    - [Material for nbdev](guides/guides_1.md)
    - [Material for nbdev](guides/guides_2.md)
"""

        assert actual == expected, actual

settings.ini created.
Directory /tmp/tmpn8t6zf6m/mkdocs created.
File '/tmp/tmpn8t6zf6m/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpn8t6zf6m/mkdocs/summary_template.txt' generated.


/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')
/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')


- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- [Sample](sample.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Material for nbdev](blogs/blogs_1.md)
    - [Material for nbdev](blogs/blogs_2.md)
- guides
    - [Material for nbdev](guides/guides_1.md)
    - [Material for nbdev](guides/guides_2.md)



In [ ]:
# | export


def _generate_summary_for_sidebar(
    root_path: str,
    cache_path: Path
) -> str:
    """Generate a summary for the sidebar

    Args:
        root_path: The root path of the project.
        cache_path: The path to the _proc directory.

    Returns:
        The summary for the sidebar

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        sidebar = _read_sidebar_from_yml(root_path)
        expanded_sidebar = _expand_sidebar_if_needed(root_path, sidebar)
        sidebar_nav = _generate_nav_from_sidebar(expanded_sidebar, cache_path)

        return sidebar_nav

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    for dir_name in ["guides", "blogs"]:
        (Path(d) / "nbs" / dir_name).mkdir(parents=True)
        for i in [f"{dir_name}_1", f"{dir_name}_2"]:
            shutil.copyfile(
                Path(d) / "nbs" / "index.ipynb",
                (Path(d) / "nbs" / f"{dir_name}" / f"{i}.ipynb"),
            )

    cmd = f'cd "{d}" && nbdev_sidebar'
    print(f"executing the command: {cmd}")
    _sprun(cmd)

    new(d)
    
    with set_cwd(d):
        cache_path = proc_nbs()

    actual = _generate_summary_for_sidebar(d, cache_path)

    print(actual)

    expected = """- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Material for nbdev](blogs/blogs_1.md)
    - [Material for nbdev](blogs/blogs_2.md)
- guides
    - [Material for nbdev](guides/guides_1.md)
    - [Material for nbdev](guides/guides_2.md)
"""
    assert actual == expected, actual

settings.ini created.
executing the command: cd "/tmp/tmpi2xpcj_v" && nbdev_sidebar
Directory /tmp/tmpi2xpcj_v/mkdocs created.
File '/tmp/tmpi2xpcj_v/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpi2xpcj_v/mkdocs/summary_template.txt' generated.


/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')
/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')


- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Material for nbdev](blogs/blogs_1.md)
    - [Material for nbdev](blogs/blogs_2.md)
- guides
    - [Material for nbdev](guides/guides_1.md)
    - [Material for nbdev](guides/guides_2.md)



In [ ]:
# | export


def _copy_not_found_file_and_get_path(root_path: str, file_prefix: str) -> str:
    """Copy the CLI command found file to the docs directory and return the path to the file.

    Args:
        root_path: The root path of the project
        file_prefix: The prefix of the file to be copied

    Returns:
        The path to the copied file

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = get_root_data_path() / f"{file_prefix}_not_found.md"
    if not src_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {src_path.resolve()} does not exists!"
        )

    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True, parents=True)
    dst_path = docs_path / f"{file_prefix}_not_found.md"
    shutil.copyfile(src_path, dst_path)

    return (
        f"({dst_path.name})"
        if file_prefix == "changelog"
        else " " * 4 + f"- [Not found]({dst_path.name})"
    )

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    new(d)

    cli_summary = _copy_not_found_file_and_get_path(d, "cli_commands")
    print(cli_summary)

    _dst_path = Path(d) / "mkdocs" / "docs"
    assert cli_summary == " " * 4 + "- [Not found](cli_commands_not_found.md)"
    assert (_dst_path / "cli_commands_not_found.md").exists()

settings.ini created.
Directory /tmp/tmp3zuqttcn/mkdocs created.
File '/tmp/tmp3zuqttcn/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp3zuqttcn/mkdocs/summary_template.txt' generated.
    - [Not found](cli_commands_not_found.md)


### Build API

In [ ]:
# | export


def _get_submodules(package_name: str) -> List[str]:
    """Get all submodules of a package.

    Args:
        package_name: The name of the package.

    Returns:
        A list of submodules.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        m = importlib.import_module(package_name)
    except ModuleNotFoundError as e:
        if (
            "NBDEV_MKDOCS_PATCH_IMPORTLIB" in os.environ
            and os.environ["NBDEV_MKDOCS_PATCH_IMPORTLIB"] != "false"
        ):
            # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
            m = importlib.import_module("nbdev_mkdocs")
        else:
            raise e
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules

In [ ]:
os.environ["NBDEV_MKDOCS_PATCH_IMPORTLIB"] = "true"

submodules = _get_submodules("repo")
submodules

['repo.docstring', 'repo.mkdocs', 'repo.social_image_generator']

In [ ]:
# # | export


# def _generate_api_doc_for_submodule(
#     root_path: str, docs_dir_name: str, submodule: str
# ) -> str:
#     """Generate API documentation for a submodule.

#     Args:
#         root_path: The root path of the project
#         docs_dir_name: The name of the docs directory
#         submodule: The submodule for which to generate the API documentation

#     Returns:
#         The path to the generated API documentation

#     !!! note

#         The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
#     """
#     subpath = f"{docs_dir_name}/" + submodule.replace(".", "/") + ".md"
#     path = Path(root_path) / "mkdocs" / "docs" / subpath
#     path.parent.mkdir(exist_ok=True, parents=True)
#     with open(path, "w") as f:
#         f.write(f"::: {submodule}")
#     subnames = submodule.split(".")
#     if len(subnames) > 2:
#         return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
#     else:
#         return f"- [{submodule}]({subpath})"



In [ ]:
# with TemporaryDirectory() as d:
#     root_path = Path(d)
#     docs_dir_name="numpy"
#     submodule="numpy.array"
    
#     actual = _generate_api_doc_for_submodule(root_path, docs_dir_name, submodule)
#     print(actual)
    

In [ ]:
# | export

def _import_submodules(module_name: str) -> List[types.ModuleType]:
    def import_module(name: str) -> Optional[types.ModuleType]:
        try:
            # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
            return importlib.import_module(name)
        except Exception:
            return None
        
    package_names = _get_submodules(module_name)
    modules = [import_module(n) for n in package_names]
    return [m for m in modules if m is not None]

In [ ]:
numpy_submodels = _import_submodules("numpy")

/usr/lib/python3.8/pkgutil.py:92: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  __import__(info.name)


WARN: Could not locate executable icc
WARN: Could not locate executable ecc


In [ ]:
actual = _import_submodules("nbdev_mkdocs")

expected = [
    "nbdev_mkdocs.docstring",
    "nbdev_mkdocs.mkdocs",
    "nbdev_mkdocs.social_image_generator",
]
display(actual)
assert [x.__name__ for x in actual] == expected

[<module 'nbdev_mkdocs.docstring' from '/home/davor/work/projects/airt/nbdev-mkdocs/nbdev_mkdocs/docstring.py'>,
 <module 'nbdev_mkdocs.mkdocs' from '/home/davor/work/projects/airt/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py'>,
 <module 'nbdev_mkdocs.social_image_generator' from '/home/davor/work/projects/airt/nbdev-mkdocs/nbdev_mkdocs/social_image_generator.py'>]

In [ ]:
# | export
def _import_functions_and_classes(
    m: types.ModuleType,
) -> List[Tuple[str, Union[types.FunctionType, Type[Any]]]]:
    return [(x, y) for x, y in getmembers(m) if isfunction(y) or isclass(y)]

In [ ]:
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))

[('AbstractChildWatcher', asyncio.unix_events.AbstractChildWatcher),
 ('AbstractEventLoop', asyncio.events.AbstractEventLoop),
 ('AbstractEventLoopPolicy', asyncio.events.AbstractEventLoopPolicy),
 ('AbstractServer', asyncio.events.AbstractServer),
 ('BaseEventLoop', asyncio.base_events.BaseEventLoop)]

In [ ]:
# | export

def _is_private(name: str) -> bool:
    parts = name.split(".")
    return any([part.startswith("_") for part in parts])

In [ ]:
assert _is_private('asyncio.base_events._SendfileFallbackProtocol')
assert not _is_private('asyncio.base_events.SendfileFallbackProtocol')

In [ ]:
# | export


def _import_all_members(module_name: str) -> List[str]:
    submodules = _import_submodules(module_name)
    members: List[Tuple[str, Union[types.FunctionType, Type[Any]]]] = list(
        itertools.chain(*[_import_functions_and_classes(m) for m in submodules])
    )

    names = [f"{y.__module__}.{y.__name__}" for x, y in members]
    names = [
        name for name in names if not _is_private(name) and name.startswith(module_name)
    ]
    return names

In [ ]:
members = _import_all_members("numpy")
members[:5], members[-5:]

(['numpy.array_api.linalg.matmul',
  'numpy.array_api.linalg.matrix_transpose',
  'numpy.array_api.linalg.tensordot',
  'numpy.array_api.linalg.vecdot',
  'numpy.array_api.linalg.EighResult'],
 ['numpy.typing.tests.test_typing.test_code_runs',
  'numpy.typing.tests.test_typing.test_extended_precision',
  'numpy.typing.tests.test_typing.test_fail',
  'numpy.typing.tests.test_typing.test_reveal',
  'numpy.typing.tests.test_typing.test_success'])

In [ ]:
# | export

def _add_all_submodules(members: List[str]) -> List[str]:
    def _f(x: str) -> List[str]:
        xs = x.split(".")
        return [".".join(xs[:i])+"." for i in range(1, len(xs))]
    
    submodules = list(set(itertools.chain(*[_f(x) for x in members])))
    members = members + submodules
    members = sorted(members)
    return members

In [ ]:
members = ["a.b", "a.c.d", "a.e.d"]
expected = ['a.', 'a.b', 'a.c.', 'a.c.d', 'a.e.', 'a.e.d']
actual = _add_all_submodules(members)
assert actual == expected, actual

In [ ]:
members = ["a.b", "a.c", "a.e.d", "a.e.f"]
expected = ['a.', 'a.b', 'a.c', 'a.e.', 'a.e.d', 'a.e.f']
actual = _add_all_submodules(members)
assert actual == expected

In [ ]:
# | export

def _get_api_summary_item(x: str) -> str:
    xs = x.split('.')
    if x.endswith("."):
        indent = " " * (4 * (len(xs)-1))
        return f"{indent}- {xs[-2]}"
    else:
        indent = " " * (4 * (len(xs)))
        return f"{indent}- [{xs[-1]}](api/{'/'.join(xs)}.md)"

In [ ]:
assert _get_api_summary_item("a.b.") == " " * 8 + "- b"
assert _get_api_summary_item("a.b") == " " * 8 + "- [b](api/a/b.md)"

In [ ]:
# | export

def _get_api_summary(members: List[str]) -> str:
    return "\n".join([_get_api_summary_item(x) for x in members]) + "\n"

In [ ]:
members = ['a.', 'a.b', 'a.c.', 'a.c.d', 'a.e.', 'a.e.f']
actual = _get_api_summary(members)
expected = """    - a
        - [b](api/a/b.md)
        - c
            - [d](api/a/c/d.md)
        - e
            - [f](api/a/e/f.md)
"""
print("*"*100)
print("- API")
print(actual, end="")
print("*"*100)
assert actual == expected

****************************************************************************************************
- API
    - a
        - [b](api/a/b.md)
        - c
            - [d](api/a/c/d.md)
        - e
            - [f](api/a/e/f.md)
****************************************************************************************************


In [ ]:
# | export

def _generate_api_doc(name: str, docs_path: Path) -> Path:
    xs = name.split(".")
    module_name = ".".join(xs[:-1])
    member_name = xs[-1]
    path = docs_path / f"{('/').join(xs)}.md"
    content = f"""`{module_name}.{member_name}`

::: {module_name}
    options:
        members: ["{member_name}"]
"""
    
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(content)
        
    return path

In [ ]:
with TemporaryDirectory() as d:
    docs_path = Path(d)
    path = _generate_api_doc('a.b', docs_path)
    
    assert path == docs_path / "a" / "b.md"
    
    with open(path, "r") as f:
        actual = f.read()
        
    expected = """`a.b`

::: a
    options:
        members: ["b"]
"""
    
    assert actual == expected


In [ ]:
# | export


def _generate_api_docs(members: List[str], docs_path: Path) -> List[Path]:
    return [_generate_api_doc(x, docs_path) for x in members if not x.endswith(".")]

In [ ]:
members = ['a.', 'a.b', 'a.c.', 'a.c.d', 'a.e.', 'a.e.d']

with TemporaryDirectory() as d:
    docs_path = Path(d)
    generated_paths = _generate_api_docs(members, docs_path)
    
    expected = [docs_path / "a/b.md", docs_path / "a/c/d.md", docs_path / "a/e/d.md"]
    assert generated_paths == expected

In [ ]:
# | export

def _generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    """Generate API documentation for a module.

    Args:
        root_path: The root path of the project.
        module_name: The name of the module.

    Returns:
        A string containing the API documentation for the module.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    members = _import_all_members(module_name)
    members_with_submodules = _add_all_submodules(members)
    
    api_summary = _get_api_summary(members_with_submodules)

    _generate_api_docs(members_with_submodules, Path(root_path) / "mkdocs" / "docs" / "api")

    return api_summary
#     return textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
@contextmanager
def add_tmp_path_to_sys_path(dir_):
    """Add a temporary path to sys.path

    Args:
        dir_ : the path to add to sys.path

    Returns:
        None

    Raises:
        ValueError: If dir_ is None

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    dir_ = Path(dir_).absolute().resolve(strict=True)
    original_path = sys.path[:]
    sys.path.insert(0, str(dir_))
    try:
        yield
    finally:
        sys.path = original_path


with TemporaryDirectory() as d:
    module_name = "nbdev_mkdocs"

    expected = \
"""    - nbdev_mkdocs
        - docstring
            - [run_examples_from_docstring](api/nbdev_mkdocs/docstring/run_examples_from_docstring.md)
        - mkdocs
            - [nbdev_mkdocs_docs](api/nbdev_mkdocs/mkdocs/nbdev_mkdocs_docs.md)
            - [new](api/nbdev_mkdocs/mkdocs/new.md)
            - [prepare](api/nbdev_mkdocs/mkdocs/prepare.md)
            - [preview](api/nbdev_mkdocs/mkdocs/preview.md)
        - social_image_generator
            - [generate_social_image](api/nbdev_mkdocs/social_image_generator/generate_social_image.md)
"""
    api_summary = _generate_api_docs_for_module(d, module_name)
    print("*"*100)
    print("- API")
    print(api_summary)
    print("*"*100)
    
    assert api_summary == expected
    
    files = sorted([x for x in (Path(d) / "mkdocs" / "docs" / "api").glob("**/*") if x.is_file()])
    
    display(files)
    
filenames_1 = re.findall(r"[a-z|A-Z|_]*.md", api_summary)
filenames_2 = [f.name for f in files]
assert set(filenames_1) == set(filenames_2)

****************************************************************************************************
- API
    - nbdev_mkdocs
        - docstring
            - [run_examples_from_docstring](api/nbdev_mkdocs/docstring/run_examples_from_docstring.md)
        - mkdocs
            - [nbdev_mkdocs_docs](api/nbdev_mkdocs/mkdocs/nbdev_mkdocs_docs.md)
            - [new](api/nbdev_mkdocs/mkdocs/new.md)
            - [prepare](api/nbdev_mkdocs/mkdocs/prepare.md)
            - [preview](api/nbdev_mkdocs/mkdocs/preview.md)
        - social_image_generator
            - [generate_social_image](api/nbdev_mkdocs/social_image_generator/generate_social_image.md)

****************************************************************************************************


[Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/docstring/run_examples_from_docstring.md'),
 Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/mkdocs/nbdev_mkdocs_docs.md'),
 Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/mkdocs/new.md'),
 Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/mkdocs/prepare.md'),
 Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/mkdocs/preview.md'),
 Path('/tmp/tmpcnrt_pry/mkdocs/docs/api/nbdev_mkdocs/social_image_generator/generate_social_image.md')]

### CLI

In [ ]:
# | export


def _restrict_line_length(s: str, width: int = 80) -> str:
    """Restrict the line length of a string.

    Args:
        s: The string to be processed.
        width: The maximum line length.

    Returns:
        A new string in which each line is less than the specified width.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _s = ""

    for blocks in s.split("\n\n"):
        sub_block = blocks.split("\n  ")
        for line in sub_block:
            line = line.replace("\n", " ")
            line = "\n".join(textwrap.wrap(line, width=width, replace_whitespace=False))
            if len(sub_block) == 1:
                _s += line + "\n\n"
            else:
                _s += "\n" + line + "\n" if line.endswith(":") else " " + line + "\n"
    return _s

In [ ]:
s = """usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory in the **root_path** directory and populates it with
initial values. You should edit mkdocs.yml file to customize it if needed.

positional arguments:
  root_path

optional arguments:
  -h, --help  show this help message and exit show this help message and exit show this help message and exit
  -h, --help  show this help message and exit
  --port PORT
  --domain DOMAIN
"""

width = 60
doc = _restrict_line_length(s, width)

print(doc)
assert all([len(line) <= width for line in doc.splitlines() if line.strip() != ""])

usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory
in the **root_path** directory and populates it with initial
values. You should edit mkdocs.yml file to customize it if
needed.


positional arguments:
 root_path

optional arguments:
 -h, --help  show this help message and exit show this help
message and exit show this help message and exit
 -h, --help  show this help message and exit
 --port PORT
 --domain DOMAIN



In [ ]:
# | export


def _generate_cli_doc_for_submodule(
    root_path: str, docs_dir_name: str, cmd: str
) -> str:
    """Generate CLI documentation for a submodule.

    Args:
        root_path: The root path of the project.
        docs_dir_name: The name of the directory where the documentation will be stored.
        cmd: The command to generate documentation for.

    Returns:
        The generated documentation.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cli_app_name = cmd.split("=")[0]
    module_name = cmd.split("=")[1].split(":")[0]
    method_name = cmd.split("=")[1].split(":")[1]

    subpath = f"{docs_dir_name}/{cli_app_name}.md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)

    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        m = importlib.import_module(module_name)
        if isinstance(getattr(m, method_name), typer.Typer):
            cli_doc = generate_cli_doc(module_name, cli_app_name)
        else:
            cmd = f"{cli_app_name} --help"
            cli_doc = (
                # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
                subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
                    cmd,
                    shell=True,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                ).stdout.decode("utf-8")
            )

            cli_doc = _restrict_line_length(cli_doc)
            cli_doc = "\n```\n" + cli_doc + "\n```\n"

    except AttributeError as e:
        cli_doc = f"Unable to generate documentation for command. Execution of `{cli_app_name} --help` command failed."

    with open(path, "w") as f:
        f.write(cli_doc)

    return f"- [{cli_app_name}]({subpath})"


def _generate_cli_docs_for_module(root_path: str, module_name: str) -> str:
    """Generate CLI docs for a module.

    Args:
        root_path: The root path of the module
        module_name: The name of the module

    Returns:
        The generated CLI docs

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    docs_dir_name = "cli"
    shutil.rmtree(
        Path(root_path) / "mkdocs" / "docs" / f"{docs_dir_name}", ignore_errors=True
    )
    console_scripts = get_value_from_config(root_path, "console_scripts")

    if not console_scripts:
        ret_val = _copy_not_found_file_and_get_path(
            root_path=root_path, file_prefix="cli_commands"
        )
        return ret_val

    submodule_summary = "\n".join(
        [
            _generate_cli_doc_for_submodule(
                root_path=root_path, docs_dir_name=docs_dir_name, cmd=cmd
            )
            for cmd in console_scripts.split("\n")
            if cmd != ""
        ]
    )

    return textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    # update settings.ini file add invalid console script
    updater = ConfigUpdater()
    updater.read(settings_path)
    updater["DEFAULT"]["console_scripts"] = "nbdev_mkdocs=nbdev_mkdocs._cli:_app"
    updater["DEFAULT"]["console_scripts"].append(
        "invalid_cmd=nbdev_mkdocs._cli:_invalid_app"
    )
    updater.update_file()

    new(d)

    _module = "repo"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)
    expected = """    - [nbdev_mkdocs](cli/nbdev_mkdocs.md)
    - [invalid_cmd](cli/invalid_cmd.md)"""

    assert cli_summary == expected, cli_summary

    _dst_path = Path(d) / "mkdocs" / "docs"
    assert (_dst_path / "cli" / "nbdev_mkdocs.md").exists()
    assert (_dst_path / "cli" / "invalid_cmd.md").exists()

    with (_dst_path / "cli" / "invalid_cmd.md").open("r") as f:
        contents = f.read()

    print(contents)
    assert (
        contents
        == "Unable to generate documentation for command. Execution of `invalid_cmd --help` command failed."
    )

settings.ini created.
Directory /tmp/tmp6c3n4fvg/mkdocs created.
File '/tmp/tmp6c3n4fvg/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp6c3n4fvg/mkdocs/summary_template.txt' generated.
    - [nbdev_mkdocs](cli/nbdev_mkdocs.md)
    - [invalid_cmd](cli/invalid_cmd.md)
Unable to generate documentation for command. Execution of `invalid_cmd --help` command failed.


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    settings_path = Path(d) / "settings.ini"
    new(d)

    _module = "invalid_module"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)

    _dst_path = Path(d) / "mkdocs" / "docs"

    for path in _dst_path.iterdir():
        print(path)

    assert cli_summary == " " * 4 + "- [Not found](cli_commands_not_found.md)"

    assert (_dst_path / "cli_commands_not_found.md").exists()

settings.ini created.
Directory /tmp/tmp46ual30i/mkdocs created.
File '/tmp/tmp46ual30i/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp46ual30i/mkdocs/summary_template.txt' generated.
    - [Not found](cli_commands_not_found.md)
/tmp/tmp46ual30i/mkdocs/docs/cli_commands_not_found.md


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    _module = "nbdev_mkdocs"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)

    # make sure all paths and content exist
    paths = re.findall("\(.*?\)", cli_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path
        assert "cli" in str(path)

settings.ini created.
Directory /tmp/tmp3ecj56g7/mkdocs created.
File '/tmp/tmp3ecj56g7/mkdocs/mkdocs.yml' generated.
File '/tmp/tmp3ecj56g7/mkdocs/summary_template.txt' generated.
    - [nbdev_mkdocs](cli/nbdev_mkdocs.md)


In [ ]:
# | export


def _copy_change_log_if_exists(root_path: str, docs_path: Union[Path, str]) -> str:
    """Copy the CHANGELOG.md file to the docs folder if it's not already present.

    Args:
        root_path: The root path of the project.
        docs_path: The path to the docs folder.

    Returns:
        The path to the copied CHANGELOG.md file.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    source_change_log_path = Path(root_path) / "CHANGELOG.md"
    dst_change_log_path = Path(docs_path) / "CHANGELOG.md"

    if source_change_log_path.exists():
        shutil.copy(source_change_log_path, dst_change_log_path)
        changelog = "(CHANGELOG.md)"
    else:
        changelog = _copy_not_found_file_and_get_path(
            root_path=root_path, file_prefix="changelog"
        )

    return changelog

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "(changelog_not_found.md)"

    change_log_path = Path(d) / "CHANGELOG.md"
    with open(change_log_path, "w") as f:
        f.write("CHANGELOG")

    (Path(d) / "mkdocs" / "docs").mkdir(exist_ok=True, parents=True)
    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "(CHANGELOG.md)"

settings.ini created.
Directory /tmp/tmpneaa5fes/mkdocs created.
File '/tmp/tmpneaa5fes/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpneaa5fes/mkdocs/summary_template.txt' generated.
change_log=(changelog_not_found.md)
change_log=(CHANGELOG.md)


### Bringing it all together

In [ ]:
# | export


def _build_summary(
    root_path: str,
    module: str,
    cache_path: Path,
) -> None:
    # create docs_path if needed
    """Create a summary navigation file for generating navigation that is compatible with mkdocs.

    Args:
        root_path: The root path of the project.
        module: The module to generate the API documentation for.
        cache_path: The path to the _proc directory.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    # generate markdown files
    _generate_markdown_from_files(root_path, cache_path)

    # copy images to docs dir and update path in generated markdown files
    _copy_images_to_docs_dir(root_path, cache_path)

    # generates sidebar navigation
    sidebar = _generate_summary_for_sidebar(root_path, cache_path)

    # generate API
    api = _generate_api_docs_for_module(root_path, module)

    # generate CLI
    cli = _generate_cli_docs_for_module(root_path, module)

    # copy CHANGELOG.md as CHANGELOG.md is exists
    changelog = _copy_change_log_if_exists(root_path, docs_path)

    # read summary template from file
    with open(Path(root_path) / "mkdocs" / "summary_template.txt") as f:
        summary_template = f.read()

    summary = summary_template.format(
        sidebar=sidebar, api=api, cli=cli, changelog=changelog
    )
    summary = "\n".join(
        [l for l in [l.rstrip() for l in summary.split("\n")] if l != ""]
    )

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

In [ ]:
_test_summary_template = """{sidebar}
- Reference
{api}
- Command line
{cli}
- [Change log]{changelog}
"""

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["custom_sidebar"] = False
    updater.update_file()

    copy_guides(_root_path, d)
    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    _summary_template_path = Path(d) / "mkdocs" / "summary_template.txt"
    _summary_template_path.unlink()
    with _summary_template_path.open("w", encoding="utf-8") as f:
        f.write(_test_summary_template)

    with set_cwd(d):
        cache_path = proc_nbs()
#     _build_summary(d, "repo", cache_path)
    _build_summary(d, "nbdev_mkdocs", cache_path)

    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)
    assert "- [Material for nbdev](index.md)" in summary
    assert "- [Change log](CHANGELOG.md)" in summary
    assert "- Reference\n" in summary
    assert "- Command line\n" in summary

settings.ini created.
../nbs/guides/Setting_Up_Document_Versioning.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Setting_Up_Document_Versioning.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Auto_Generating_Docstrings.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Auto_Generating_Docstrings.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Setting_up_social_cards.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Setting_up_social_cards.ipynb
../nbs/guides/Adding_Guides.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Adding_Guides.ipynb
../nbs/guides/Adding_Release_Notes.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Adding_Release_Notes.ipynb
../nbs/guides/Advanced_Customization_Options.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Advanced_Customization_Options.ipynb
../nbs/guides/Handling_Pandas_In_The_Output.ipynb, /tmp/tmpuw9xe76x/nbs/guides/Handling_Pandas_In_The_Output.ipynb
../nbs/guides/images/cli_not_found.png, /tmp/

/home/davor/.local/lib/python3.8/site-packages/nbdev/processors.py:186: UserWarning: Found cells containing imports and other code. See FAQ.
---
import asyncio

xys = _import_functions_and_classes(asyncio)
display(xys[:5])

actual = [x for x, _ in xys]

assert set(["run", "sleep", "BaseEventLoop"]).issubset(set(actual))
---

  warn(f'Found cells containing imports and other code. See FAQ.\n---\n{cell.source}\n---\n')


- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- blogs
    - [Adding guides](blogs/Adding_Guides.md)
    - [Adding release notes](blogs/Adding_Release_Notes.md)
    - [Advanced customization options](blogs/Advanced_Customization_Options.md)
    - [Auto-generating docstrings](blogs/Auto_Generating_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Handling pandas in the output](blogs/Handling_Pandas_In_The_Output.md)
    - [Setting up document versioning](blogs/Setting_Up_Document_Versioning.md)
    - [Setting up social cards](blogs/Setting_up_social_cards.md)
- guides
    - [Adding guides](guides/Adding_Guides.md)
    - [Adding release notes](guides/Adding_Release_Notes.md)
    - [Advanced customization options](guides/Advanced_Customization_Options.md)
    - [Auto-generating docstrings](guides/Auto_Generating_Docstrings.md)
   

### Copy CNAME if needed

In [ ]:
# | export


def _copy_cname_if_needed(root_path: str) -> None:
    """Copy the CNAME file to mkdocs/docs/CNAME if it's not already present.

    Args:
        root_path: The root path of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cname_path = Path(root_path) / "CNAME"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "CNAME"
    if cname_path.exists():
        dst_path.parent.mkdir(exist_ok=True, parents=True)
        shutil.copyfile(cname_path, dst_path)
        typer.secho(
            f"File '{cname_path.resolve()}' copied to '{dst_path.resolve()}'.",
        )
    else:
        typer.secho(
            f"File '{cname_path.resolve()}' not found, skipping copying..",
        )

In [ ]:
for has_cname in [True, False]:
    with TemporaryDirectory() as d:
        settings_path = Path(d) / "settings.ini"
        for fname in ["settings.ini", "README.md"] + ["CNAME"] if has_cname else []:
            shutil.copyfile(Path("..") / fname, Path(d) / fname)

        _copy_cname_if_needed(d)
        if has_cname:
            assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
        else:
            assert not (Path(d) / "mkdocs" / "docs" / "CNAME").exists()

File '/tmp/tmppgsbjpk5/CNAME' copied to '/tmp/tmppgsbjpk5/mkdocs/docs/CNAME'.
File '/tmp/tmp1wk4aq8z/CNAME' not found, skipping copying..


In [ ]:
# | export


def _copy_docs_overrides(root_path: str) -> None:
    """Copy the docs_overrides directory to the mkdocs/docs/overrides directory.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = Path(root_path) / "mkdocs" / "docs_overrides"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "overrides"

    if not src_path.exists():
        raise_error_and_exit(
            f"Unexpected error: path {src_path.resolve()} does not exists!"
        )

    shutil.rmtree(dst_path, ignore_errors=True)
    shutil.copytree(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    assert (Path(d) / "mkdocs" / "docs_overrides").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js" / "extra.js").exists()

    _copy_docs_overrides(d)

    assert (Path(d) / "mkdocs" / "docs" / "overrides").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js" / "extra.js").exists()

settings.ini created.
Directory /tmp/tmpqpjrzpp3/mkdocs created.
File '/tmp/tmpqpjrzpp3/mkdocs/mkdocs.yml' generated.
File '/tmp/tmpqpjrzpp3/mkdocs/summary_template.txt' generated.


In [ ]:
# | export

def _get_backtick_enclosed_string(s: str) -> str:
    """Get the string enclosed in backticks.

    Args:
        s: The string to extract from

    Returns:
        The extracted string enclosed in backticks.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    pattern = r"`(.*?)`"
    match = re.search(pattern, s)
    return match.group(1) # type: ignore

In [ ]:
fixture = "[`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
expected = "nbdev_mkdocs.mkdocs.prepare"
actual = _get_backtick_enclosed_string(fixture)
display(actual)
assert actual == expected

fixture = '[`google`](https://www.google.com/)'
expected = "google"
actual = _get_backtick_enclosed_string(fixture)
display(actual)
assert actual == expected

fixture = '[`airt.client.Client.get_token`](https://docs.airt.ai/api_client.html#client.get_token)'
expected = "airt.client.Client.get_token"
actual = _get_backtick_enclosed_string(fixture)
display(actual)
assert actual == expected

'nbdev_mkdocs.mkdocs.prepare'

'google'

'airt.client.Client.get_token'

In [ ]:
# | export

def _get_sym_path_from_nbdev_lookup(nbdev_lookup: NbdevLookup, v: Tuple[str, str, str]) -> str: # type: ignore
    """Get the symbol path from the NbdevLookup instance

    Args:
        nbdev_lookup: Instance of NbdevLookup
        v: The value to look for

    Returns:
        The matched symbol path

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    py_syms = merge(*L(o['syms'].values() for o in nbdev_lookup.entries.values()).concat())
    ret_val: List[str] = [key for key, value in py_syms.items() if value == v]
    return ret_val[0]

In [ ]:
c = NbdevLookup(incl_libs="nbdev")
fixture = "nbdev.doclinks.NbdevLookup.linkify"
actual = _get_sym_path_from_nbdev_lookup(c, c[fixture])
print(actual)

assert actual == fixture

c = NbdevLookup(incl_libs="nbdev-mkdocs")
fixture = "nbdev_mkdocs.mkdocs.prepare"
actual = _get_sym_path_from_nbdev_lookup(c, c[fixture])
print(actual)

assert actual == fixture

nbdev.doclinks.NbdevLookup.linkify
nbdev_mkdocs.mkdocs.prepare


In [ ]:
# | export

def _get_current_docs_version(docs_versioning: str, lib_version: str) -> str:
    """Get the current docs version.

    Args:
        docs_versioning: The value set for docs_versioning flag in settings.ini file.
        lib_version: The current version of the library.

    Returns:
        The current docs version.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    return (
        ".".join(lib_version.split(".")[:-1])
        if docs_versioning == "minor" and lib_version.replace(".", "").isdigit()
        else lib_version
    )

In [ ]:
docs_versioning = "minor"
lib_version = "2.3.4"
actual = _get_current_docs_version(docs_versioning, lib_version)
expected = "2.3"

print(actual)
assert actual == expected

docs_versioning = "patch"
lib_version = "1.0.1"
actual = _get_current_docs_version(docs_versioning, lib_version)
expected = "1.0.1"

print(actual)
assert actual == expected


docs_versioning = "minor"
lib_version = "1.0.1dev"
actual = _get_current_docs_version(docs_versioning, lib_version)
expected = "1.0.1dev"

print(actual)
assert actual == expected

docs_versioning = "patch"
lib_version = "1.0.1rc0"
actual = _get_current_docs_version(docs_versioning, lib_version)
expected = "1.0.1rc0"

print(actual)
assert actual == expected

2.3
1.0.1
1.0.1dev
1.0.1rc0


In [ ]:
# | export


def _fix_sym_links(s: str, nbdev_lookup: NbdevLookup, docs_versioning: str, lib_version: str) -> str:  # type: ignore
    """Fix the default sym links generated by nbdev in the given string.

    Args:
        s: The string to fix
        nbdev_lookup: Instance of the NbdevLookup class.
        docs_versioning: The value set for docs_versioning flag in settings.ini file.
        lib_version: The current version of the library.

    Returns:
        The string with correct links added to the symbol references.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    pattern = r"\[`.+?`\]\(https?://[^)]+\)"
    for match in re.findall(pattern, s):
        symbol = _get_backtick_enclosed_string(match)
        symbol_details = nbdev_lookup[symbol]
        if symbol_details is not None:
            fixed_part = "/".join(match.split("/")[:4])
            package_name = symbol_details[1].split("/")[0]
            fixed_part_with_docs_version = (
                f"{fixed_part}/{_get_current_docs_version(docs_versioning, lib_version)}"
                if docs_versioning != "" and docs_versioning != "None"
                else fixed_part
            )
            sym_path = _get_sym_path_from_nbdev_lookup(nbdev_lookup, symbol_details)
            updated_link = f"{fixed_part_with_docs_version}/{package_name}_api_docs/{package_name}/{symbol_details[1].split('/')[1].replace('.py', '')}/#{sym_path})"
            s = s.replace(match, updated_link)
    return s

In [ ]:
nbdev_lookup = NbdevLookup(incl_libs="nbdev")

fixture = "This is a link to a symbol: [`nbdev.doclinks.NbdevLookup.linkify`](https://nbdev.fast.ai/api/doclinks.html#nbdevlookup.linkify)"
docs_versioning = "None"
lib_version = "0.1.1"
expected = "This is a link to a symbol: [`nbdev.doclinks.NbdevLookup.linkify`](https://nbdev.fast.ai/api/nbdev_api_docs/nbdev/doclinks/#nbdev.doclinks.NbdevLookup.linkify)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected

'This is a link to a symbol: [`nbdev.doclinks.NbdevLookup.linkify`](https://nbdev.fast.ai/api/nbdev_api_docs/nbdev/doclinks/#nbdev.doclinks.NbdevLookup.linkify)'

In [ ]:
nbdev_lookup = NbdevLookup(incl_libs="nbdev-mkdocs")

fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = ""
lib_version = "0.1.1"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected

fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = "None"
lib_version = "0.1.1"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected

fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = "minor"
lib_version = "0.1.1"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/0.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/0.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected


fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = "minor"
lib_version = "0.1.1rc0"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/0.1.1rc0/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/0.1.1rc0/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected


fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = "patch"
lib_version = "1.2.12"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected


fixture = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)"
docs_versioning = "patch"
lib_version = "1.2.12dev"
expected = "This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)"       
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected


'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/0.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/0.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/0.1.1rc0/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/0.1.1rc0/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

'This is a link to a symbol: [`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare) and this is also a link [`prepare`](https://airtai.github.io/nbdev-mkdocs/1.2.12dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)'

In [ ]:
fixture = "This is not a link to a symbol: [`google`](https://www.google.com/)"
docs_versioning = "None"
lib_version = "0.1.1"

expected = "This is not a link to a symbol: [`google`](https://www.google.com/)"
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected

fixture = "This is a link to an external symbol: [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array)"
docs_versioning = "None"
lib_version = "0.1.1"

expected = "This is a link to an external symbol: [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array)"
actual = _fix_sym_links(fixture, nbdev_lookup, docs_versioning, lib_version)
display(actual)
assert actual == expected

'This is not a link to a symbol: [`google`](https://www.google.com/)'

'This is a link to an external symbol: [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array)'

In [ ]:
#| export

def _fix_sym_links_in_nbs(root_path: str, cache_path: Path, nbdev_lookup: NbdevLookup, docs_versioning: str, lib_version: str) -> None: # type: ignore
    """Fix the default sym links generated by nbdev in the notebooks

    Args:
        root_path: The root path of the project.
        cache_path: The path to the _proc directory.
        nbdev_lookup: Instance of NbdevLookup.
        docs_versioning: The value set for docs_versioning flag in settings.ini file.
        lib_version: The current version of the library.


    !!! note

        The above docstring is autogenerated by docstring-gen library (https://docstring-gen.airt.ai)
    """
    files = [
        f
        for f in cache_path.rglob("*")
        if f.suffix == ".ipynb"
        and not any(cache_path.startswith(".") for cache_path in f.parts)
        and not f.name.startswith("_")
    ]
    
    for file in files:
        _f = nbformat.read(file, as_version=4)
        for cell in _f.cells:
            if cell.cell_type == "markdown":
                updated_src = _fix_sym_links(cell["source"], nbdev_lookup, docs_versioning, lib_version)
                cell["source"] = updated_src

        nbformat.write(_f, file)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    shutil.copyfile(_root_path / "fixtures/Test_Sym_Links_In_Docs.ipynb", Path(d) / "nbs/Test_Sym_Links_In_Docs.ipynb")

        
    with set_cwd(d):
        with unset_env_var("IN_TEST"):
            nbdev_export.__wrapped__()
        cache_path = proc_nbs()
    
    !cat {d}/repo/_modidx.py
    
    cmd = f'cd "{d}" && nbdev_docs'
    _sprun(cmd)
    
    nbdev_lookup = NbdevLookup(incl_libs = "nbdev-mkdocs")
    docs_versioning = "None"
    lib_version = "0.1.1"
    _fix_sym_links_in_nbs(d, cache_path, nbdev_lookup, docs_versioning, lib_version)
    
    with open(f"{d}/_proc/Test_Sym_Links_In_Docs.ipynb", "r") as f:
        file_contents = f.read()
    
    assert "[`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)" not in file_contents
    assert "[`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare)" in file_contents
    
    
    nbdev_lookup = NbdevLookup(incl_libs = "nbdev")
    docs_versioning = "None"
    lib_version = "0.1.1"
    _fix_sym_links_in_nbs(d, cache_path, nbdev_lookup, docs_versioning, lib_version)
    
    with open(f"{d}/_proc/Test_Sym_Links_In_Docs.ipynb", "r") as f:
        file_contents = f.read()
    
    assert "[`nbdev.doclinks.NbdevLookup.linkify`](https://nbdev.fast.ai/api/doclinks.html#nbdevlookup.linkify)" not in file_contents
    assert "[`nbdev.release.changelog`](https://nbdev.fast.ai/api/release.html#changelog)" not in file_contents
    
    assert "[`nbdev.doclinks.NbdevLookup.linkify`](https://nbdev.fast.ai/api/nbdev_api_docs/nbdev/doclinks/#nbdev.doclinks.NbdevLookup.linkify)" in file_contents
    
    print("OK")
    
#     !cat {d}/_proc/Test_Sym_Links_In_Docs.ipynb
    
    

settings.ini created.
# Autogenerated by nbdev

d = { 'settings': { 'branch': 'branch',
                'doc_baseurl': '/repo',
                'doc_host': 'https://user.github.io',
                'git_url': 'https://github.com/user/repo',
                'lib_path': 'repo'},
  'syms': {}}
OK


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
expected = ["https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare", "https://airtai.github.io/nbdev-mkdocs/0.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare", "https://airtai.github.io/nbdev-mkdocs/0.1.1/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare"]  

for i, docs_versioning in enumerate([None, "minor", "patch"]):
    with TemporaryDirectory() as d:
        run_nbdev_new(d)
        updater = ConfigUpdater()
        updater.read(Path(d) / "settings.ini")
        updater["DEFAULT"]["version"].add_after.option("docs_versioning", docs_versioning)
        updater["DEFAULT"]["version"] = "0.1.1"
        updater.update_file()
        
        _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
        shutil.copyfile(_root_path / "fixtures/Test_Sym_Links_In_Docs.ipynb", Path(d) / "nbs/Test_Sym_Links_In_Docs.ipynb")


        with set_cwd(d):
            with unset_env_var("IN_TEST"):
                nbdev_export.__wrapped__()
            cache_path = proc_nbs()

        cmd = f'cd "{d}" && nbdev_docs'
        _sprun(cmd)

        nbdev_lookup = NbdevLookup(incl_libs = "nbdev-mkdocs")
        docs_versioning = get_value_from_config(d, "docs_versioning")
        lib_version = get_value_from_config(d, "version")
        
        _fix_sym_links_in_nbs(d, cache_path, nbdev_lookup, docs_versioning, lib_version)

        with open(f"{d}/_proc/Test_Sym_Links_In_Docs.ipynb", "r") as f:
            file_contents = f.read()

        assert "[`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)" not in file_contents
        assert expected[i] in file_contents, f"{expected[i]}, {file_contents}"
        
        print("OK.")

settings.ini created.


/home/davor/.local/lib/python3.8/site-packages/configupdater/option.py:115: NoneValueDisallowed: Cannot represent <docs_versioning = None>, it will be converted to <docs_versioning = ''>.
    Please use ``allow_no_value=True`` with ``ConfigUpdater``.
    
  NoneValueDisallowed.warn(self._key)


OK.
settings.ini created.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


OK.
settings.ini created.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


OK.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
expected = ["https://airtai.github.io/nbdev-mkdocs/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare", "https://airtai.github.io/nbdev-mkdocs/0.1.1dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare", "https://airtai.github.io/nbdev-mkdocs/0.1.1dev/nbdev_mkdocs_api_docs/nbdev_mkdocs/mkdocs/#nbdev_mkdocs.mkdocs.prepare"]  


for i, docs_versioning in enumerate([None, "minor", "patch"]):
    with TemporaryDirectory() as d:
        run_nbdev_new(d)
        updater = ConfigUpdater()
        updater.read(Path(d) / "settings.ini")
        updater["DEFAULT"]["version"].add_after.option("docs_versioning", docs_versioning)
        updater["DEFAULT"]["version"] = "0.1.1dev"
        updater.update_file()
        
        _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
        shutil.copyfile(_root_path / "fixtures/Test_Sym_Links_In_Docs.ipynb", Path(d) / "nbs/Test_Sym_Links_In_Docs.ipynb")


        with set_cwd(d):
            with unset_env_var("IN_TEST"):
                nbdev_export.__wrapped__()
            cache_path = proc_nbs()

        cmd = f'cd "{d}" && nbdev_docs'
        _sprun(cmd)

        nbdev_lookup = NbdevLookup(incl_libs = "nbdev-mkdocs")
        docs_versioning = get_value_from_config(d, "docs_versioning")
        lib_version = get_value_from_config(d, "version")

        _fix_sym_links_in_nbs(d, cache_path, nbdev_lookup, docs_versioning, lib_version)

        with open(f"{d}/_proc/Test_Sym_Links_In_Docs.ipynb", "r") as f:
            file_contents = f.read()

        assert "[`nbdev_mkdocs.mkdocs.prepare`](https://airtai.github.io/nbdev-mkdocs/mkdocs.html#prepare)" not in file_contents
        assert expected[i] in file_contents, f"{expected[i]}, {file_contents}"
        
        print("OK.")

settings.ini created.


/home/davor/.local/lib/python3.8/site-packages/configupdater/option.py:115: NoneValueDisallowed: Cannot represent <docs_versioning = None>, it will be converted to <docs_versioning = ''>.
    Please use ``allow_no_value=True`` with ``ConfigUpdater``.
    
  NoneValueDisallowed.warn(self._key)


OK.
settings.ini created.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


OK.
settings.ini created.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


OK.


/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
# | export


def nbdev_mkdocs_docs(root_path: str, refresh_quarto_settings: bool = False) -> None:
    """Prepare mkdocs documentation

    Args:
        root_path: The root path of the project
        refresh_quarto_settings: Flag to refresh quarto yml file. This flag should be set to `True`
            if this function is called directly without calling prepare.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        if refresh_quarto_settings:
            refresh_quarto_yml()

        _copy_cname_if_needed(root_path)

        _copy_docs_overrides(root_path)

        lib_name = get_value_from_config(root_path, "lib_name")
        lib_path = get_value_from_config(root_path, "lib_path")

        cache_path = proc_nbs()
        nbdev_lookup = NbdevLookup(incl_libs=lib_name.replace("_", "-"))
        docs_versioning = get_value_from_config(root_path, "docs_versioning")
        lib_version = get_value_from_config(root_path, "version")
        _fix_sym_links_in_nbs(
            root_path, cache_path, nbdev_lookup, docs_versioning, lib_version
        )

        _build_summary(root_path, lib_path, cache_path)

        cmd = f"mkdocs build -f \"{(Path(root_path) / 'mkdocs' / 'mkdocs.yml').resolve()}\""
        _sprun(cmd)


def prepare(root_path: str, no_test: bool = False) -> None:
    """Prepare mkdocs for serving

    Args:
        root_path: The root path of the project
        no_test: If set to False, the unit tests will be run, else they will be skipped

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        if no_test:
            nbdev_export.__wrapped__()
            refresh_quarto_yml()
            nbdev_readme.__wrapped__(chk_time=True)
        else:
            cmd = "nbdev_prepare"
            _sprun(cmd)

    nbdev_mkdocs_docs(root_path)

In [ ]:
with patch('__main__.importlib') as mock:
    mock.return_value = nbdev
    assert importlib() is nbdev
    print(importlib())

<module 'nbdev' from '/home/davor/.local/lib/python3.8/site-packages/nbdev/__init__.py'>


In [ ]:
# | notest

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    updater = ConfigUpdater()
    updater.read(settings_path)
    updater["DEFAULT"]["version"] = "5.0.1"
    updater.update_file()
    
#     !cat {settings_path}

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)

    for f in ["CNAME", "nbs/Docstring.ipynb"]:
        shutil.copyfile(_root_path / f, Path(d) / f)

    new(d)
    #     !cat {d}/mkdocs/mkdocs.yml

    prepare(d)

    assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "guides").exists()

settings.ini created.
../nbs/guides/Setting_Up_Document_Versioning.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Setting_Up_Document_Versioning.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Auto_Generating_Docstrings.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Auto_Generating_Docstrings.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Setting_up_social_cards.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Setting_up_social_cards.ipynb
../nbs/guides/Adding_Guides.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Adding_Guides.ipynb
../nbs/guides/Adding_Release_Notes.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Adding_Release_Notes.ipynb
../nbs/guides/Advanced_Customization_Options.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Advanced_Customization_Options.ipynb
../nbs/guides/Handling_Pandas_In_The_Output.ipynb, /tmp/tmptzw0cxjt/nbs/guides/Handling_Pandas_In_The_Output.ipynb
../nbs/guides/images/cli_not_found.png, /tmp/

/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


## Preview

In [ ]:
# | export


def preview(root_path: str, port: Optional[int] = None) -> None:
    """Preview the mkdocs documentation.

    Args:
        root_path: The root path of the documentation.
        port: The port to serve the documentation on.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        prepare(root_path=root_path, no_test=True)

        cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
        if port:
            cmd = cmd + f":{port}"

        with subprocess.Popen(  # nosec B603:subprocess_without_shell_equals_true
            shlex.split(cmd),
            stdout=subprocess.PIPE,
            bufsize=1,
            text=True,
            universal_newlines=True,
        ) as p:
            for line in p.stdout:  # type: ignore
                print(line, end="")

        if p.returncode != 0:
            typer.secho(
                f"Command cmd='{cmd}' failed!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(6)

In [ ]:
# | notest

_test_summary_template = """{sidebar}
- Reference
{api}
- Command line
{cli}
- [Change log]{changelog}
"""

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    updater = ConfigUpdater()
    updater.read(settings_path)
    updater["DEFAULT"]["version"] = "1.0.1"
    updater.update_file()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    copy_guides(_root_path, d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    _summary_template_path = Path(d) / "mkdocs" / "summary_template.txt"
    _summary_template_path.unlink()
    with _summary_template_path.open("w", encoding="utf-8") as f:
        f.write(_test_summary_template)

    preview(d, port=4000)

settings.ini created.
../nbs/guides/Setting_Up_Document_Versioning.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Setting_Up_Document_Versioning.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Auto_Generating_Docstrings.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Auto_Generating_Docstrings.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Setting_up_social_cards.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Setting_up_social_cards.ipynb
../nbs/guides/Adding_Guides.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Adding_Guides.ipynb
../nbs/guides/Adding_Release_Notes.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Adding_Release_Notes.ipynb
../nbs/guides/Advanced_Customization_Options.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Advanced_Customization_Options.ipynb
../nbs/guides/Handling_Pandas_In_The_Output.ipynb, /tmp/tmpm8igqjfs/nbs/guides/Handling_Pandas_In_The_Output.ipynb
../nbs/guides/images/cli_not_found.png, /tmp/

/usr/local/lib/python3.8/dist-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


AssertionError: 